In [107]:
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'dmivashkin@gmail.com'
path_to_credentials = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-ivashkin-23/Theme_1/karpovapi-37ae06984e71.json'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credentials, scope)
gs = gspread.authorize(credentials)

table_name = 'Тестовое Дмитрий Ивашкин'
sheet = gs.create(table_name)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/%s" % sheet.id

# Make it visible to other guys
sheet.share(my_mail, perm_type='user', role='writer')


# Данные первого месяца
option_a = {'Clicks': 25130, 'Installations': 1560, 'Orders': 345, 'AOV': 8425}
option_b = {'Clicks': 23750, 'Installations': 1345, 'Orders': 285, 'AOV': 10875}

df = pd.DataFrame([option_a, option_b], index=['Option A', 'Option B'])

df['CR'] = round(df.Orders/df.Clicks, 3)

df['Sales'] = df.AOV*df.Orders

best_banner = df['CR'].idxmax()
top_sales = df.loc['Option B', 'Sales'] > df.loc['Option A', 'Sales']

sheet = 'Первый месяц'
d2g.upload(df, table_name, sheet, credentials=credentials, row_names=True)

print('Данные первого месяца:')
print(df)

print(f'\nЛучшим вариантом баннера в первый месяц оказался: {best_banner}')
print(f'Метриками, показывающими это, являются CR (конверсия в покупку) и абсолютное значение покупок')

print(f'\nПодтвердили гипотезу о том, что повышенная скидка на первый заказ приведёт к большему количеству продаж в первый месяц среди пользователей IPhone: {top_sales}')

# Данные второго и третьего месяца
return_rate_a = 0.24
return_rate_b = 0.07

AOV_a = df.loc['Option A', 'AOV'] / 2
AOV_b = df.loc['Option B', 'AOV'] / 2

Orders_a = df.loc['Option A', 'Installations'] * return_rate_a
Orders_b = df.loc['Option B', 'Installations'] * return_rate_b

df.loc['Option A', 'Orders'] += Orders_a + (Orders_a * 0.05)
df.loc['Option B', 'Orders'] += Orders_b + (Orders_b * 0.02)

df.loc['Option A', 'AOV'] = AOV_a
df.loc['Option B', 'AOV'] = AOV_b

df['CR'] = round(df.Orders/df.Clicks, 3)

df['Sales'] = round(df.AOV*df.Orders)

sheet = 'Три месяца'
d2g.upload(df, table_name, sheet, credentials=credentials, row_names=True)

best_banner = df['CR'].idxmax()

print('\nДанные по трём месяцам:')
print(df)

print(f'\nБолее ценными пользователями для маркетплейса совокупно (по CR и продажам) являются пользователи: {best_banner}')

print('\nСсылка на таблицу:')
print(spreadsheet_url)

Данные первого месяца:
          Clicks  Installations  Orders    AOV     CR    Sales
Option A   25130           1560     345   8425  0.014  2906625
Option B   23750           1345     285  10875  0.012  3099375

Лучшим вариантом баннера в первый месяц оказался: Option A
Метриками, показывающими это, являются CR (конверсия в покупку) и абсолютное значение покупок

Подтвердили гипотезу о том, что повышенная скидка на первый заказ приведёт к большему количеству продаж в первый месяц среди пользователей IPhone: True

Данные по трём месяцам:
          Clicks  Installations   Orders     AOV     CR      Sales
Option A   25130           1560  738.120  4212.5  0.029  3109330.0
Option B   23750           1345  381.033  5437.5  0.016  2071867.0

Ссылка на таблицу:
https://docs.google.com/spreadsheets/d/1B_bV2gW7tVi8-HabGUlpC9Jpo44MAXRmGW4LA5b0JB8

Более ценными пользователями для маркетплейса совокупно (по CR и продажам) являются пользователи: Option A
